<h2>Collaborative Based Recommendation System</h2>

Although there is no room for a collaborative recommendation system with this single dataset, let us play around with the data and explore this model.

In [55]:
import numpy as np
import pandas as pd

In [57]:
# Reading the csv file
df=pd.read_csv('C:\\Users\\CHETAN\\OneDrive\\Desktop\\PES1UG20CS109\\DA\\PROJECT\\netflix_titles.csv',index_col=0)

In [58]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity


In [59]:
from numpy import random
import datetime
from numpy import dot
from numpy import random
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors  

In [60]:
# Filling the null values with mode for the numeric column and then the columns with string is filled with NULL
df['country'] = df['country'].fillna(df['country'].mode()[0])
df['date_added'] = df['date_added'].fillna(df['date_added'].mode()[0])
df['rating'] = df['rating'].fillna(df['country'].mode()[0])
df['duration'] = df['duration'].fillna('0 min')
df['cast'] = df['cast'].fillna('NULL')
df['director'] = df['director'].fillna('NULL')

In [61]:
# Checking is any Missing data in each column of the dataset.
df.isna().sum()

type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [62]:


df.dropna(subset=['director','cast','country','date_added','rating'],axis=0,inplace=True)
df.drop(['description'],axis=1,inplace=True)
print()
print("Details after dropping null rows and columns")
print("Total Rows: {}\nTotal Columns: {}".format(df.shape[0],df.shape[1]))

dfDatabase = df.copy()

print()
print("Database Created")


Details after dropping null rows and columns
Total Rows: 8807
Total Columns: 10

Database Created


In [63]:
!pip install unidecode


In [64]:
import unidecode
df['cast'] = df['cast'].apply(lambda x : unidecode.unidecode(x))

df['date_added'] = df['date_added'].apply(lambda x : x[-4:])
df['date_added'] = df['date_added'].astype('int64')
df['show_age'] = 2021 - df['date_added']
df['show_age'] = df['show_age'].apply(lambda x : str(x)+" years")

dfVis = df.copy()

df.drop(['date_added','release_year'],axis=1,inplace=True)

print("Data Procesing Done")

Data Procesing Done


In [65]:
Begin = datetime.datetime.now()

userid = np.arange(1, 2000+1, 1)
userid = ["uid" + str(userid) for userid in userid]

usercols = df['title'].unique()
userwatch = random.randint(5, size=(2000,len(usercols)))

watchhist = pd.DataFrame(userwatch,columns=usercols)
watchhist.insert(0,"user_id",userid)
watchhist = watchhist.set_index('user_id')
watchhist = watchhist.transpose()
watchhist.index.name = 'title'

watchhist.head()

user_id,uid1,uid2,uid3,uid4,uid5,uid6,uid7,uid8,uid9,uid10,...,uid1991,uid1992,uid1993,uid1994,uid1995,uid1996,uid1997,uid1998,uid1999,uid2000
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,1,3,0,2,3,4,4,0,0,4,...,1,4,0,1,4,1,3,2,2,4
Blood & Water,4,4,2,0,4,0,3,2,0,1,...,1,0,1,4,4,4,2,2,1,3
Ganglands,4,0,0,0,2,3,1,2,0,0,...,0,0,3,3,4,4,1,2,1,1
Jailbirds New Orleans,4,2,0,2,4,4,0,0,4,1,...,1,1,4,0,0,0,1,4,4,0
Kota Factory,1,4,2,3,4,2,4,3,2,1,...,1,1,3,0,4,2,0,3,2,1


In [66]:
sparselist = csr_matrix(watchhist.values)
watchhist.reset_index(inplace=True)

knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
knn.fit(sparselist)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [67]:
def collab_recommendation_system(title):
    
    try:
        row = watchhist[watchhist['title'] == title].index[0]

        distances , indices = knn.kneighbors(sparselist[row],n_neighbors=5+1) 
        indices = [l for m in indices.tolist() for l in m]
        distances = [l for m in distances.tolist() for l in m]

        recommendation = {}

        for i in range(1,len(indices)):
            movie_name = watchhist.iloc[indices[i]]['title']
            recommendation[movie_name] = distances[i]

        recommendation = {k: v for k, v in sorted(recommendation.items(), key=lambda item: item[1])}
        results = pd.DataFrame()

        for key,value in recommendation.items():
            row = dfDatabase[dfDatabase['title'] == key]
            results = results.append(row)

        return(results.iloc[1:,:-1])
        
    except:
        print("Incorrect Movie Title,try another one.")   

In [68]:
collab_recommendation_system("Rocky IV")   

,type,title,director,cast,country,date_added,release_year,rating,duration
show_id,,,,,,,,,
s5549,Movie,Life 2.0,Jason Spingarn-Koff,NULL,United States,"April 1, 2017",2010,TV-MA,99 min
s2671,TV Show,Hasmukh,NULL,"Vir Das, Ranvir Shorey, Manoj Pahwa, Ravi Kish...",India,"April 17, 2020",2020,TV-MA,1 Season
s659,TV Show,"Nevertheless,",NULL,"Song Kang, Han So-hee, Chae Jong-hyeop, Lee Yu...",United States,"June 20, 2021",2021,TV-14,1 Season
s2316,Movie,The F**k-It List,Michael Duggan,"Eli Brown, Madison Iseman, Marcus Scribner, Je...",United States,"July 1, 2020",2019,TV-MA,103 min


In [74]:
collab_recommendation_system("Stranger Things")

,type,title,director,cast,country,date_added,release_year,rating,duration
show_id,,,,,,,,,
s8187,Movie,The Age of Shadows,Jee-woon Kim,"Kang-ho Song, Gong Yoo, Ji-min Han, Tae-goo Eo...",South Korea,"January 1, 2018",2016,TV-MA,140 min
s2228,TV Show,Cursed,NULL,"Katherine Langford, Devon Terrell, Gustaf Skar...","United Kingdom, Australia","July 17, 2020",2020,TV-MA,1 Season
s6918,TV Show,Hannibal,NULL,"Hugh Dancy, Mads Mikkelsen, Caroline Dhavernas...",United States,"June 6, 2020",2015,TV-MA,3 Seasons
s474,Movie,Born to Play,Viridiana Lieberman,NULL,United States,"July 12, 2021",2020,TV-MA,93 min


In [70]:
collab_recommendation_system("PK")

,type,title,director,cast,country,date_added,release_year,rating,duration
show_id,,,,,,,,,
s3657,TV Show,Rookie Historian Goo Hae-Ryung,NULL,"Shin Sae-kyeong, Cha Eun-woo, Park Ki-woong, L...",South Korea,"July 18, 2019",2019,TV-14,1 Season
s3650,Movie,The Twins Effect II,"Corey Yuen, Patrick Leung","Donnie Yen, Jaycee Chan, Charlene Choi, Gillia...","Hong Kong, China","July 21, 2019",2004,TV-14,106 min
s4392,Movie,Halkaa,Nila Madhab Panda,"Tathastu, Aryan Preet, Ranvir Shorey, Paoli Da...",India,"November 15, 2018",2018,TV-14,111 min
s2773,TV Show,Wassup Man GO!,NULL,Joon Park,United States,"March 27, 2020",2020,TV-MA,1 Season


In [75]:
collab_recommendation_system("RRR")

Incorrect Movie Title,try another one.


In [71]:
End = datetime.datetime.now()
RunTime = round((End - Begin).total_seconds()/60.0)
print("Final Run Time' was {} Mins".format(RunTime))  

Final Run Time' was 0 Mins
